### Import dependencies (LangGraph is a graph framework with nodes, edges and acts as state machine)

In [ ]:
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.messages import convert_to_openai_messages, convert_to_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import random
import ast
import inspect
import instructor
import json

from utils.utils import get_tool_descriptions, format_ai_message


### Single Node Graph

In [ ]:
## Single state to start off - The variables are part of this state - Schema for the state
class State(BaseModel):
    message: str
    answer: str = ""
    vibe: str

In [ ]:
def append_vibes_to_query(state: State) -> dict:

    return {
        "answer": f"{state.message} {state.vibe}"
    }


In [ ]:
workflow = StateGraph(State)

workflow.add_node("append_vibes_to_query", append_vibes_to_query)

workflow.add_edge(START, "append_vibes_to_query")
workflow.add_edge("append_vibes_to_query", END)

graph = workflow.compile()


In [ ]:

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
initial_state = {
    "message": "Give me some vibes!",
    "answer": "abc",
    "vibe": "I'm feeling like a badass today!"
}


In [ ]:
result = graph.invoke(initial_state)


### Whatever is added to graph computation in the state, is mutated to the state {Answer is overwritten}

In [ ]:
result

### Conditional Graph

In [ ]:
class State(BaseModel):
    message: str
    answer: str = ""

In [ ]:
def append_vibes_to_query(state: State) -> dict:

    return {
        "answer": state.message
    }


In [ ]:
def router(state: State) -> Literal["append_vibe_1", "append_vibe_2", "append_vibe_3"]:

    vibes = ["append_vibe_1", "append_vibe_2", "append_vibe_3"]

    vibe_path = random.choice(vibes)

    return vibe_path

In [ ]:
def append_vibe_1(state: State) -> dict:

    vibe = "I'm feeling like a badass today!"

    return {
        "answer": f"{state.answer} {vibe}"
    }

def append_vibe_2(state: State) -> dict:

    vibe = "I'm feeling like a boss today!"

    return {
        "answer": f"{state.answer} {vibe}"
    }

def append_vibe_3(state: State) -> dict:

    vibe = "I'm feeling like a legend today!"

    return {
        "answer": f"{state.answer} {vibe}"
    }


In [ ]:
workflow = StateGraph(State)

workflow.add_node("append_vibes_to_query", append_vibes_to_query)
workflow.add_node("append_vibe_1", append_vibe_1)
workflow.add_node("append_vibe_2", append_vibe_2)
workflow.add_node("append_vibe_3", append_vibe_3)


workflow.add_conditional_edges(
    ## Node
    "append_vibes_to_query",
    ## function to get value for the condition
    router,
    ## If router's value matches node name, no need to have this block below.
    ## But if it doesn't match, we need to explicitly state the mapping
    {
        "append_vibe_1": "append_vibe_1",
        "append_vibe_2": "append_vibe_2",
        "append_vibe_3": "append_vibe_3"
    }
)

workflow.add_edge(START, "append_vibes_to_query")

workflow.add_edge("append_vibe_1", END)
workflow.add_edge("append_vibe_2", END)
workflow.add_edge("append_vibe_3", END)

graph = workflow.compile()

In [ ]:

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
## set initial state -- This state progagates, mutable across execution of nodes.
initial_state = {
    "message": "I am here to add some vibes:",
}


In [ ]:

result = graph.invoke(initial_state)

In [ ]:
result

In [ ]:

result = graph.invoke(initial_state)

In [ ]:
result

### Agent Graph ==> Agent that can call "tools". Tool, in our vocabulary, is a function given to Agent which can decide if to call or not

In [ ]:

## We need to define what it does, Args, Returns for agent to understand and call the tool
def append_vibes(query: str, vibe: str) -> str:
    """Takes in a query and a vibe and returns a string with the query and vibe appended.

    Args:
        query: The query to append the vibe to.
        vibe: The vibe to append to the query.

    Returns:
        A string with the query and vibe appended.
    """
    
    return f"{query} {vibe}"



In [ ]:
## returns the function metadata in proper json format
get_tool_descriptions([append_vibes])

In [ ]:
## A ToolCall class
class ToolCall(BaseModel):
    name: str
    arguments: dict

## Result from agent
class AgentResponse(BaseModel):
    ## Answer
    answer: str
    ## List of tool calls Agent did
    tool_calls: List[ToolCall] = Field(default_factory=list)

## The state that is muteed foe every agent call, Reasoning and answering
class State(BaseModel):
    ## Iteration of messages -- all messages in the conversation
    messages: Annotated[List[Any], add] = []
    ## The latest message from the agent
    message: str = ""
    iteration: int = 0
    ## The answer of the agent
    answer: str = ""
    ## List of available tools
    available_tools: List[Dict[str, Any]] = []
    ## List of tool calls -- What agent has called
    tool_calls: List[ToolCall] = []

### Computation Function ==> Agent node

In [ ]:
def agent_node(state: State) -> dict:

   prompt_template =  """You are an assistant that is generating vibes for a user.

You will be given a selection of tools you can use to add vibes to a user's query.

<Available tools>
{{ available_tools | tojson }}
</Available tools>

When you need to use a tool, format your response as:

<tool_call>
{"name": "tool_name", "arguments": {...}}
</tool_call>

Instructions:
- You need to use the tools to add vibes to the user's query.
- Add a random vibe to the user's query.
"""

   template = Template(prompt_template)
   
   prompt = template.render(
      available_tools=state.available_tools
   )

   client = instructor.from_openai(OpenAI())

   response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        response_model=AgentResponse,
        messages=[{"role": "system", "content": prompt}, {"role": "user", "content": state.message}],
        temperature=0.5,
   )

   ai_message = format_ai_message(response)

   return {
      "messages": [ai_message],
      "tool_calls": response.tool_calls
   }

### Short explainer on message types and formatting

In [ ]:
## This is Open AI compatible message format
conversation  = [
    {'role': 'user', 'content': 'Give me some vibes!'},
    {'role': 'assistant', 'content': 'I am here to add some vibes:'},
    {'role': 'user', 'content': 'I am feeling like a badass today!'}
]

In [ ]:
## This converts Open AI compatible message format to LangChain message format
convert_to_messages(conversation)

In [ ]:
## Just to check, if reconversion is right, this tells that our Open AI compatible message is right.
convert_to_openai_messages(convert_to_messages(conversation))

In [ ]:
dummy_response = AgentResponse(answer="I am here to add some vibes:", tool_calls=[ToolCall(name="append_vibes", arguments={"query": "Give me some vibes!", "vibe": "I am feeling like a badass today!"})])

In [ ]:
dummy_response

In [ ]:
## This util class is used to convert to Lang chain compatible message format
format_ai_message(dummy_response)

### End of Short explainer on message types and formatting

In [ ]:
## Conditional pattern before Agent decides to follow a particular path
def tool_router(state: State) -> str:
    """Decide whether to continue or end"""
    
    ## Bec we have only one tool
    if state.iteration > 1:
        return "end"
    elif len(state.tool_calls) > 0:
        return "tools"
    else:
        return "end"

In [ ]:
## The entire graph construction for Agentic node execution based on conditionals and available tools
workflow = StateGraph(State)

## Add all tools to "tools"
tools = [append_vibes]
## ToolNode is a helper class in LangGraph, we create a list of functions.
## We reoute to ToolNode. The Graph looks for a AIMessage of type langchain's type with ToolCalls list at the end of conversation history
## If it finds this kind of message, it executes the tools
tool_node = ToolNode(tools)
### They will be injected into prompt of the agent.
tool_descriptions = get_tool_descriptions(tools)

## Add the agent node that has the prompt, computation
workflow.add_node("agent_node", agent_node)
## Add the tool node
workflow.add_node("tool_node", tool_node)
## START edge
workflow.add_edge(START, "agent_node")


workflow.add_conditional_edges(
    "agent_node",
    tool_router,
    {
        ## There are tool calls to execute
        "tools": "tool_node",
        ## End because no tool calls are availabe to execute
        "end": END
    }
)

workflow.add_edge("tool_node", END)

graph = workflow.compile()

In [ ]:

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:

initial_state = {
    "message": "Give me some vibes!",
    "available_tools": tool_descriptions
}
result = graph.invoke(initial_state)

In [ ]:
result

### Agent Graph with Loopback from Tools (ReAct Agent)

In [ ]:
def append_vibes(query: str, vibe: str) -> str:
    """Takes in a query and a vibe and returns a string with the query and vibe appended.

    Args:
        query: The query to append the vibe to.
        vibe: The vibe to append to the query.

    Returns:
        A string with the query and vibe appended.
    """
    
    return f"{query} {vibe}"

In [ ]:

class ToolCall(BaseModel):
    name: str
    arguments: dict

class AgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)

class State(BaseModel):
    ## Append current message to existing messages
    messages: Annotated[List[Any], add] = []
    message: str = ""
    iteration: int = 0
    answer: str = ""
    available_tools: List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = []



In [ ]:
def agent_node(state: State) -> dict:

   prompt_template =  """You are a assistant that is generating vibes for a user.

You will be given a selection of tools you can use to add vibes to a user's query.

<Available tools>
{{ available_tools | tojson }}
</Available tools>

When you need to use a tool, format your response as:

<tool_call>
{"name": "tool_name", "arguments": {...}}
</tool_call>

Instructions:
- You need to use the tools to add vibes to the user's query.
- Add a random vibe to the user's query.
- You must return a tool call in the first interaction.
"""

   template = Template(prompt_template)
   
   prompt = template.render(
      available_tools=state.available_tools
   )

   messages = state.messages

   conversation = convert_to_openai_messages(messages)

   client = instructor.from_openai(OpenAI())

   response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        response_model=AgentResponse,
        messages=[{"role": "system", "content": prompt}, *conversation],
        temperature=0.5,
   )

   ai_message = format_ai_message(response)

   ## This is dictionary of key, values
   return {
      ## This is a list, got changed, append all messages
      "messages": [ai_message],
      "tool_calls": response.tool_calls,
      "iteration": state.iteration + 1,
      ## latest message
      "answer": response.answer
   }

In [ ]:
def tool_router(state: State) -> str:
    """Decide whether to continue or end"""
    
    if state.iteration > 1:
        return "end"
    elif len(state.tool_calls) > 0:
        return "tools"
    else:
        return "end"



In [ ]:
workflow = StateGraph(State)

tools = [append_vibes]
tool_node = ToolNode(tools)
tool_descriptions = get_tool_descriptions(tools)

workflow.add_node("agent_node", agent_node)
workflow.add_node("tool_node", tool_node)

workflow.add_edge(START, "agent_node")

workflow.add_conditional_edges(
    "agent_node",
    tool_router,
    {
        "tools": "tool_node",
        "end": END
    }
)

## circular loop til the conditional edge tells when to exit
workflow.add_edge("tool_node", "agent_node")

graph = workflow.compile()


In [ ]:

display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:

initial_state = {
    "message": "Give me some vibes!",
    "available_tools": tool_descriptions
}
result = graph.invoke(initial_state)


In [ ]:
result